In [1]:
from src.motion_models.trajectory_generation.route_generation import *
from src.visual.visual_tools import *

import numpy as np

%load_ext autoreload
%autoreload 2

### Classifying CV with different velocity noise + CT + CA

In [2]:
from src.evaluation_tools.trajectory_classifier import classify_confusion_matrix
from src.filters import CVKalmanFilter, CTKalmanFilter, CAKalmanFilter, InitializationData

import numpy as np

dt = 0.04
OBSERVATION_NOISE = np.array([0.2,0.2])
T = 400
TRAJECTORIES_PER_CLASS = 10
initial_position = np.array([0,0])

#CV1
CV1_VEL_CHANGES_STD = np.array([0.1,0.1])
v0 = np.array([4,5])

cv1_initial_trajectory_state = TrajectoryState(position=initial_position, velocity=v0)

cv1_trajectories = generate_cv_trajectory(T=T, dt=dt, initial_state=cv1_initial_trajectory_state, number_of_trajectories=10, measurement_noise_std=OBSERVATION_NOISE, vel_change_std=CV1_VEL_CHANGES_STD, seed=44)

cv1_init_data = InitializationData(observation_noise_std=OBSERVATION_NOISE, process_noise_std=CV1_VEL_CHANGES_STD)
cv1_kf = CVKalmanFilter(2, 4, 2, dt, cv1_init_data)
cv1_kf.initialize()

#CV2
CV2_VEL_CHANGES_STD = np.array([0.2,0.2])

cv2_initial_trajectory_state = TrajectoryState(position=initial_position, velocity=v0)

cv2_trajectories = generate_cv_trajectory(T=T, dt=dt, initial_state=cv2_initial_trajectory_state, number_of_trajectories=10, measurement_noise_std=OBSERVATION_NOISE, vel_change_std=CV2_VEL_CHANGES_STD, seed=45)

cv2_init_data = InitializationData(observation_noise_std=OBSERVATION_NOISE, process_noise_std=CV2_VEL_CHANGES_STD)
cv2_kf = CVKalmanFilter(2, 4, 2, dt, cv2_init_data)
cv2_kf.initialize()
#CV3
CV3_VEL_CHANGES_STD = np.array([1,1])

cv3_initial_trajectory_state = TrajectoryState(position=initial_position, velocity=v0)

cv3_trajectories = generate_cv_trajectory(T=T, dt=dt, initial_state=cv3_initial_trajectory_state, number_of_trajectories=10, measurement_noise_std=OBSERVATION_NOISE, vel_change_std=CV3_VEL_CHANGES_STD, seed=46)

cv3_init_data = InitializationData(observation_noise_std=OBSERVATION_NOISE, process_noise_std=CV3_VEL_CHANGES_STD)
cv3_kf = CVKalmanFilter(2, 4, 2, dt, cv3_init_data)
cv3_kf.initialize()

#CT
OMEGA_STD = 1e-1
omega = 1
pos_start = np.array([0,0])
ct_v0 = np.array([20, 10])
ACCL_STD = np.array([1e-1, 1e-1])

initial_generation_state = TrajectoryState(position=pos_start, velocity=ct_v0)
ct_trajectories = generate_ct_trajectory_simple(T=150, dt=dt, initial_state=initial_generation_state, omega=omega, measurement_noise_std=OBSERVATION_NOISE[0], omega_noise_std=OMEGA_STD, number_of_trajectories=TRAJECTORIES_PER_CLASS,seed=18)

ct_init = InitializationData(observation_noise_std=OBSERVATION_NOISE, process_noise_std=ACCL_STD, omega_std=OMEGA_STD)
ct_kf = CTKalmanFilter(2, 7, 2, dt=dt, initialization_data=ct_init)
ct_kf.initialize()


#CA
white_accel_density = 20
accel_noise_std = np.sqrt(dt * white_accel_density)
initial_acceleration = np.array([1, 1])

initial_trajectory_state = TrajectoryState(position=initial_position, velocity=v0, acceleration=initial_acceleration)
ca_trajectories = generate_ca_trajectory(T=150, dt=dt,initial_state=initial_trajectory_state, measurement_noise_std=OBSERVATION_NOISE, number_of_trajectories=10, seed=43, accel_noise_std=accel_noise_std)

ca_initial_matrices_data = InitializationData(observation_noise_std=OBSERVATION_NOISE, process_noise_std=OBSERVATION_NOISE, white_accel_density=white_accel_density)
ca_kf = CAKalmanFilter(2, 6, 2, dt, ca_initial_matrices_data)
ca_kf.initialize()


cm = classify_confusion_matrix(
    [ct_kf, cv1_kf, cv2_kf, cv3_kf, ca_kf],
    [ct_trajectories, cv1_trajectories, cv2_trajectories, cv3_trajectories, ca_trajectories])

print(cm)

[[10.  0.  0.  0.  0.]
 [ 0.  8.  2.  0.  0.]
 [ 0.  0. 10.  0.  0.]
 [ 0.  0.  0. 10.  0.]
 [ 0.  0.  0.  0. 10.]]


### Classifying CV, CT, CA and Singer
Note on the Singer model- based on the experiments, a shorter tau (2-3) will be classified better than a longer (10). This is probably related to the fact that a longer tau is a more gradual manuver.

In [3]:
from src.evaluation_tools.trajectory_classifier import classify_confusion_matrix
from src.filters import CVKalmanFilter, CTKalmanFilter, CAKalmanFilter, SingerKF, InitializationData

import numpy as np

dt = 0.04
OBSERVATION_NOISE = np.array([0.2,0.2])
T = 400
TRAJECTORIES_PER_CLASS = 10
initial_position = np.array([0,0])

#CV1
CV1_VEL_CHANGES_STD = np.array([0.1,0.1])
v0 = np.array([4,5])

cv1_initial_trajectory_state = TrajectoryState(position=initial_position, velocity=v0)

cv1_trajectories = generate_cv_trajectory(T=T, dt=dt, initial_state=cv1_initial_trajectory_state, number_of_trajectories=10, measurement_noise_std=OBSERVATION_NOISE, vel_change_std=CV1_VEL_CHANGES_STD, seed=44)

cv1_init_data = InitializationData(observation_noise_std=OBSERVATION_NOISE, process_noise_std=CV1_VEL_CHANGES_STD)
cv1_kf = CVKalmanFilter(2, 4, 2, dt, cv1_init_data)
cv1_kf.initialize()

#CV2
CV2_VEL_CHANGES_STD = np.array([0.2,0.2])

cv2_initial_trajectory_state = TrajectoryState(position=initial_position, velocity=v0)

cv2_trajectories = generate_cv_trajectory(T=T, dt=dt, initial_state=cv2_initial_trajectory_state, number_of_trajectories=10, measurement_noise_std=OBSERVATION_NOISE, vel_change_std=CV2_VEL_CHANGES_STD, seed=45)

cv2_init_data = InitializationData(observation_noise_std=OBSERVATION_NOISE, process_noise_std=CV2_VEL_CHANGES_STD)
cv2_kf = CVKalmanFilter(2, 4, 2, dt, cv2_init_data)
cv2_kf.initialize()

#Singer
initial_acceleration = np.array([4, 2])
tau = np.array([2, 2])
acc_std = np.array([0.4, 0.4])

initial_generation_state = TrajectoryState(position=initial_position, velocity=v0, acceleration=initial_acceleration)

singer_trajectories = generate_singer_trajectory(
    T=T,
    dt=dt,
    initial_state=initial_generation_state,
    tau=tau,
    dim=2,
    sigma_a=acc_std,
    noise_std=OBSERVATION_NOISE,
    number_of_trajectories=10,
    seed=18)

singer_init = InitializationData(
    observation_noise_std=OBSERVATION_NOISE,
    process_noise_std=acc_std)
singer_kf = SingerKF(dim=2, dt=dt, tau=tau, initialization_data=singer_init)  # tau in seconds 
singer_kf.initialize()

#CT
OMEGA_STD = 1e-1
omega = 1
pos_start = np.array([0,0])
ct_v0 = np.array([20, 10])
ACCL_STD = np.array([1e-1, 1e-1])

initial_generation_state = TrajectoryState(position=pos_start, velocity=ct_v0)
ct_trajectories = generate_ct_trajectory_simple(T=150, dt=dt, initial_state=initial_generation_state, omega=omega, measurement_noise_std=OBSERVATION_NOISE[0], omega_noise_std=OMEGA_STD, number_of_trajectories=TRAJECTORIES_PER_CLASS,seed=18)

ct_init = InitializationData(observation_noise_std=OBSERVATION_NOISE, process_noise_std=ACCL_STD, omega_std=OMEGA_STD)
ct_kf = CTKalmanFilter(2, 7, 2, dt=dt, initialization_data=ct_init)
ct_kf.initialize()


#CA
white_accel_density = 20
accel_noise_std = np.sqrt(dt * white_accel_density)
initial_acceleration = np.array([1, 1])

initial_trajectory_state = TrajectoryState(position=initial_position, velocity=v0, acceleration=initial_acceleration)
ca_trajectories = generate_ca_trajectory(T=150, dt=dt,initial_state=initial_trajectory_state, measurement_noise_std=OBSERVATION_NOISE, number_of_trajectories=10, seed=43, accel_noise_std=accel_noise_std)

ca_initial_matrices_data = InitializationData(observation_noise_std=OBSERVATION_NOISE, process_noise_std=OBSERVATION_NOISE, white_accel_density=white_accel_density)
ca_kf = CAKalmanFilter(2, 6, 2, dt, ca_initial_matrices_data)
ca_kf.initialize()


cm = classify_confusion_matrix(
    [ct_kf, cv1_kf, cv2_kf, cv3_kf, ca_kf, singer_kf],
    [ct_trajectories, cv1_trajectories, cv2_trajectories, cv3_trajectories, ca_trajectories, singer_trajectories])

print(cm)

[[10.  0.  0.  0.  0.  0.]
 [ 0.  7.  2.  0.  0.  1.]
 [ 0.  0. 10.  0.  0.  0.]
 [ 0.  0.  0. 10.  0.  0.]
 [ 0.  0.  0.  0. 10.  0.]
 [ 0.  2.  4.  0.  0.  4.]]
